In [1]:
######## # AUTO LOGIN-1 #########
account_name = 'haj'
import pandas as pd
import numpy as np
login_str = 'login_'+account_name
import threading
import importlib
login_str = 'login_'+account_name
module = importlib.import_module(login_str, package=None)
print('Logging in with account : '+str(account_name))
module.retry_autologin()

inst_token = 299011

Logging in with account : haj
Trying to login...
access token : zo0Slvacw2LNssMa6Hw0nxdfErTV3svr
request token : On5EJ3ZOhEGm5dOpQ8s2x4cghxBZFIO1


In [2]:
buy_qty_lst = []
sell_qty_lst = []
percent_lst = []
percent_diff_lst = []
log_data_lst = []
my_qty = 0

depth_data_old = pd.read_csv('./depth_data_7_5ma.csv')
del depth_data_old['Unnamed: 0']
my_qty = depth_data_old['my_qty'][depth_data_old.index.values[-1]]
depth_data_old

In [4]:
class PnLCalculator:
    def __init__(self):
        self.quantity = 0
        self.cost = 0.0
        self.market_value = 0.0
        self.r_pnl = 0.0
        self.average_price = 0.0
        self.date = 0.0

    def fill(self, n_pos, exec_price,date):
        pos_change = n_pos - self.quantity
        direction = np.sign(pos_change)
        prev_direction = np.sign(self.quantity)
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing

        new_cost = self.cost + qty_opening * exec_price
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        self.quantity = n_pos
        self.cost = new_cost
        self.date = date

    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        return self.market_value - self.cost

In [5]:
def export_calc():
    npdsf = pd.DataFrame(log_data_lst)
    npdsf = pd.concat([depth_data_old,npdsf])
    ndf =  npdsf[npdsf.trend != 0]
    
    positions_call = ndf['my_qty']
    exec_prices_call = ndf['ltp']
    bkg_and_charges = 64.5
    date_call = ndf['date']
    pnls_call = []
    pos_call = PnLCalculator()
    pnls_call = []
    for (p,e,dt) in zip(positions_call, exec_prices_call,date_call):
        pos_call.fill(p, e,dt)
        u_pnl = pos_call.update(e)
        pnls_call.append({'date':pos_call.date,
                          'Qty':int(pos_call.quantity),
                          'r_pnl':round((1000*pos_call.r_pnl),4),
                          'u_pnl':round(1000*u_pnl,4),
                          'total_pnl':round(1000*(pos_call.r_pnl+u_pnl)-(len(ndf.index.values)*(bkg_and_charges/2)),4),
                          'average_price':round(pos_call.average_price,4),
                         })

    pnls_df_call = pd.DataFrame(pnls_call)
    # print(pnls_df_call)
    # pnls_df_call['total_pnl'].plot()

    npdsf.to_csv('./depth_data_7_5ma.csv')
    pnls_df_call.to_csv('./pnls_df_call_3_5ma.csv')

In [6]:
import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)

def Average(lst): 
    return sum(lst) / len(lst) 

counter = 0

def on_ticks(ws, ticks):
    global my_qty,counter
  
    print('==========================')
    buy_quantity = ticks[0]['buy_quantity']
    sell_quantity = ticks[0]['sell_quantity']
    diff = buy_quantity - sell_quantity
    mean = (buy_quantity + sell_quantity)/2
    percent  = (diff/mean)*100
    percent_lst.append(percent)
    if(len(percent_lst)>5):
        percent_lst.pop(0)
    
    percent_10ma = Average(percent_lst)
    percent_diff = 100*(percent-percent_10ma)/percent_10ma
    print('percent_diff : '+str(round(percent_diff,2)))
    
    percent_diff_lst.append(percent_diff)
    if(len(percent_diff_lst)>5):
        percent_diff_lst.pop(0)
    percent_diff_10ma = Average(percent_diff_lst)
    print('percent_diff_10ma : '+str(round(percent_diff_10ma,2)))
    
    trend = 0
    if(percent_diff_10ma>0 and percent_diff>5):
        print('DOWN TREND')
        trend = -1
        my_qty = my_qty - 10
        
    if(percent_diff_10ma<0 and percent_diff<-5):
        print('UP TREND')    
        trend = 1
        my_qty = my_qty + 10
    
    print('my_qty : '+str(my_qty))
    
    log_data_lst.append({'buy_quantity':buy_quantity,
                         'sell_quantity':sell_quantity,
                         'diff':diff,
                         'mean':mean,
                         'percent':percent,
                         'percent_10ma':percent_10ma,
                         'percent_diff':percent_diff,
                         'percent_diff_10ma':percent_diff_10ma,
                         'trend':trend,
                         'my_qty':my_qty,
                         'ltp':ticks[0]['last_price'],
                         'date':ticks[0]['timestamp'],
    })
    
    print( 'LTP : ' + str(ticks[0]['last_price']))
#     print( 'tick : ' + str(ticks[0]))
    
    if(counter%5==0):
        export_calc()
    
    counter = counter + 1

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))
    
def on_order_update(ws, data):
    print('oder update')

module.kws.on_error = on_error
module.kws.on_noreconnect = on_noreconnect
module.kws.on_reconnect = on_reconnect
module.kws.on_order_update = on_order_update
module.kws.on_ticks = on_ticks
module.kws.on_connect = on_connect
module.kws.on_close = on_close

module.kws.connect()

percent_diff : 0.0
percent_diff_10ma : 0.0
my_qty : 0
LTP : 74.7325
percent_diff : 0.34
percent_diff_10ma : 0.17
my_qty : 0
LTP : 74.7325
percent_diff : 0.77
percent_diff_10ma : 0.37
my_qty : 0
LTP : 74.7325
percent_diff : 0.57
percent_diff_10ma : 0.42
my_qty : 0
LTP : 74.73
percent_diff : -3.79
percent_diff_10ma : -0.42
my_qty : 0
LTP : 74.73
percent_diff : -2.56
percent_diff_10ma : -0.78
my_qty : 0
LTP : 74.73
percent_diff : -2.2
percent_diff_10ma : -0.98
my_qty : 0
LTP : 74.73
percent_diff : -4.66
percent_diff_10ma : -1.44
my_qty : 0
LTP : 74.7275
percent_diff : -4.16
percent_diff_10ma : -1.74
my_qty : 0
LTP : 74.7275
percent_diff : -3.29
percent_diff_10ma : -1.9
my_qty : 0
LTP : 74.7275
percent_diff : -2.73
percent_diff_10ma : -2.17
my_qty : 0
LTP : 74.7275
percent_diff : -4.75
percent_diff_10ma : -2.68
my_qty : 0
LTP : 74.7225
percent_diff : -4.39
percent_diff_10ma : -3.2
my_qty : 0
LTP : 74.7225
percent_diff : -2.3
percent_diff_10ma : -3.48
my_qty : 0
LTP : 74.7225
percent_diff :

ERROR:kiteconnect.ticker:Connection error: 1006 - connection was closed uncleanly (None)
ERROR:root:closed connection on error: 1006 connection was closed uncleanly (None)
ERROR:kiteconnect.ticker:Connection closed: 1006 - connection was closed uncleanly (None)
